In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
import requests
import gmaps
import os
import math


ABS_path = "Data/ABS_Labourstatus_Data.csv"
ABS_metadata = pd.read_csv(ABS_path, low_memory = False)
ABS_metadata.head()

,AGE,Age,SEX_ABS,Sex,LFSP_C16,Labour force status,STATE,State,REGIONTYPE,Geography Level,LGA_2016,Region,TIME,Census year,Value,Flag Codes,Flags
0,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,236.0,NaN,NaN
1,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,113.0,NaN,NaN
2,A15,15 - 19,3,Persons,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,348.0,NaN,NaN
3,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional (A),2016.0,2016.0,105.0,NaN,NaN
4,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional (A),2016.0,2016.0,58.0,NaN,NaN


In [2]:
#Remove Flag Code and Flags Colums
ABS_metadata = ABS_metadata.drop(['Flag Codes', 'Flags'], axis=1)
#Remove NaN rows
ABS_metadata = ABS_metadata.dropna()
ABS_metadata.head()

,AGE,Age,SEX_ABS,Sex,LFSP_C16,Labour force status,STATE,State,REGIONTYPE,Geography Level,LGA_2016,Region,TIME,Census year,Value
0,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,236.0
1,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,113.0
2,A15,15 - 19,3,Persons,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury (C),2016.0,2016.0,348.0
3,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional (A),2016.0,2016.0,105.0
4,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional (A),2016.0,2016.0,58.0


In [3]:
new = ABS_metadata['Region'].str.split('(', n = 1, expand = True)
ABS_metadata['Region'] = new[0]
ABS_metadata.head()

,AGE,Age,SEX_ABS,Sex,LFSP_C16,Labour force status,STATE,State,REGIONTYPE,Geography Level,LGA_2016,Region,TIME,Census year,Value
0,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,236.0
1,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,113.0
2,A15,15 - 19,3,Persons,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,348.0
3,A15,15 - 19,1,Males,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional,2016.0,2016.0,105.0
4,A15,15 - 19,2,Females,1,"Employed, worked full-time",1,New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional,2016.0,2016.0,58.0


In [4]:
#State just WA
ABS_WA_data = ABS_metadata.set_index('STATE')
ABS_WA_data.head()

,AGE,Age,SEX_ABS,Sex,LFSP_C16,Labour force status,State,REGIONTYPE,Geography Level,LGA_2016,Region,TIME,Census year,Value
STATE,,,,,,,,,,,,,,
1,A15,15 - 19,1,Males,1,"Employed, worked full-time",New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,236.0
1,A15,15 - 19,2,Females,1,"Employed, worked full-time",New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,113.0
1,A15,15 - 19,3,Persons,1,"Employed, worked full-time",New South Wales,LGA2016,Local Government Areas (2016),10050,Albury,2016.0,2016.0,348.0
1,A15,15 - 19,1,Males,1,"Employed, worked full-time",New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional,2016.0,2016.0,105.0
1,A15,15 - 19,2,Females,1,"Employed, worked full-time",New South Wales,LGA2016,Local Government Areas (2016),10130,Armidale Regional,2016.0,2016.0,58.0


In [5]:

cols = [1,2,4,7,8,9,11]
ABS_WA_data.drop(ABS_WA_data.columns[cols],axis=1,inplace=True)
ABS_WA_data.head()

,AGE,Sex,Labour force status,State,Region,Census year,Value
STATE,,,,,,,
1,A15,Males,"Employed, worked full-time",New South Wales,Albury,2016.0,236.0
1,A15,Females,"Employed, worked full-time",New South Wales,Albury,2016.0,113.0
1,A15,Persons,"Employed, worked full-time",New South Wales,Albury,2016.0,348.0
1,A15,Males,"Employed, worked full-time",New South Wales,Armidale Regional,2016.0,105.0
1,A15,Females,"Employed, worked full-time",New South Wales,Armidale Regional,2016.0,58.0


In [6]:
ABS_WA_data = ABS_WA_data.reset_index()
ABS_WA_data.head()

,STATE,AGE,Sex,Labour force status,State,Region,Census year,Value
0,1,A15,Males,"Employed, worked full-time",New South Wales,Albury,2016.0,236.0
1,1,A15,Females,"Employed, worked full-time",New South Wales,Albury,2016.0,113.0
2,1,A15,Persons,"Employed, worked full-time",New South Wales,Albury,2016.0,348.0
3,1,A15,Males,"Employed, worked full-time",New South Wales,Armidale Regional,2016.0,105.0
4,1,A15,Females,"Employed, worked full-time",New South Wales,Armidale Regional,2016.0,58.0


In [9]:
District_pops = ABS_WA_data.loc[(ABS_WA_data["Labour force status"] == "Total")]
District_pops = District_pops.loc[(ABS_WA_data["AGE"] == "TOT")]
District_pops = District_pops.loc[(ABS_WA_data["Sex"] == "Persons")]
District_pops = District_pops.loc[(ABS_WA_data["State"] == "Western Australia")]
District_pops

,STATE,AGE,Sex,Labour force status,State,Region,Census year,Value
60218,5,TOT,Persons,Total,Western Australia,Albany,2016.0,29805.0
60221,5,TOT,Persons,Total,Western Australia,Armadale,2016.0,61871.0
60224,5,TOT,Persons,Total,Western Australia,Ashburton,2016.0,11305.0
60227,5,TOT,Persons,Total,Western Australia,Augusta-Margaret River,2016.0,11159.0
60230,5,TOT,Persons,Total,Western Australia,Bassendean,2016.0,12390.0
...,...,...,...,...,...,...,...,...
60620,5,TOT,Persons,Total,Western Australia,Yalgoo,2016.0,302.0
60623,5,TOT,Persons,Total,Western Australia,Yilgarn,2016.0,1046.0
60626,5,TOT,Persons,Total,Western Australia,York,2016.0,2988.0
60629,5,TOT,Persons,Total,Western Australia,No usual address,2016.0,5559.0


In [16]:
population_by_region = District_pops.groupby(District_pops["Region"]).sum()
population_by_region =pd.DataFrame(population_by_region)

In [17]:
population_by_region = population_by_region.drop(['STATE',"Census year"], axis=1)
population_by_region

,Value
Region,
Albany,29805.0
Armadale,61871.0
Ashburton,11305.0
Augusta-Margaret River,11159.0
Bassendean,12390.0
...,...
Wyalkatchem,439.0
Wyndham-East Kimberley,5511.0
Yalgoo,302.0


In [20]:
population_by_region = population_by_region.rename(columns = {"Value":"Total Persons"})
population_by_region['Total Persons']=population_by_region['Total Persons'].astype(int)
population_by_region.head()

,Total Persons
Region,
Albany,29805
Armadale,61871
Ashburton,11305
Augusta-Margaret River,11159
Bassendean,12390


In [21]:
#import region/district
path_region_district= 'Data/region_by_district_wa.csv'
region_district = pd.read_csv(path_region_district)
region_district.head()


,Region,District
0,Albany,Great Southern District
1,Armadale,Armadale District
2,Ashburton,Pilbara District
3,Augusta-Margaret River,South West District
4,Bassendean,Midland District


In [22]:
pop_by_district=pd.merge(population_by_region,region_district,left_on='Region',right_on='Region')
pop_by_district

,Region,Total Persons,District
0,Albany,29805,Great Southern District
1,Armadale,61871,Armadale District
2,Ashburton,11305,Pilbara District
3,Augusta-Margaret River,11159,South West District
4,Bassendean,12390,Midland District
...,...,...,...
134,Wyalkatchem,439,Wheatbelt District
135,Wyndham-East Kimberley,5511,Kimberley District
136,Yalgoo,302,Mid West-Gascoyne District
137,Yilgarn,1046,Wheatbelt District


In [23]:
District_pops = pop_by_district.groupby("District").sum()
District_pops

,Total Persons
District,
Armadale District,175720
Cannington District,173775
Fremantle District,210501
Goldfields-Esperance Region,42383
Great Southern District,63406
Joondalup District,269926
Kimberley District,25983
Mandurah District,207250
Mid West-Gascoyne District,48069


In [24]:
District_pops['Total Persons'].sum()

1991687

In [25]:
District_pops.to_csv("District_Population.csv")

In [ ]:
District_pops["Reference Capital"] = ["Armadale","Cannington","Fremantle","Espearance","Albany","Joondalup","Broome","Mandurah","Carnarvon","Midland","Mirrabooka","Perth City","Karratha","Bunbury","Northam"]
District_pops

In [ ]:
District_pops["Lat"] = ""
District_pops["Lng"] = ""
District_pops

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
lat = 31.9505
lng = 115.8605
params = {
    "radius": 50000,
    "key": g_key,
    "location": f"{lat},{lng}"
}


for index, row in District_pops.iterrows():
    
    keyword = row["Reference Capital"]
    
    params["query"] = f"{keyword}, Western Australia"
    
    location_info = requests.get(base_url, params=params)
    
    location_info_json = location_info.json()
    
    print(f"Searching for {keyword}")
    
    try:
        District_pops.loc[index, "Lat"] = location_info_json["results"][0]['geometry']['location']['lat']
        District_pops.loc[index, "Lng"] = location_info_json["results"][0]['geometry']['location']['lng']
    
    except (KeyError, IndexError):
        print("Not Found :(")  
    
    print("-----------------------------")
    

In [ ]:
District_pops_Indexed = District_pops.reset_index()
District_pops_Indexed

In [ ]:
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)


info_box_template = """
<dl>
<dt>District</dt><dd>{District}</dd>
<dt>Capital</dt><dd>{Reference Capital}</dd>
<dt>Total Persons</dt><dd>{Total Persons}</dd>
</dl>
"""

District_popsinfo = [info_box_template.format(**row) for index, row in District_pops_Indexed.iterrows()]
locations = District_pops_Indexed[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations, info_box_content=District_popsinfo)
                             

fig.add_layer(markers)
fig

In [ ]:
District_Totals = District_pops_Indexed.set_index('District')
District_Totals["Persons"] = District_Totals["Total Persons"] / 1000
District_Totals = District_Totals.drop(['Reference Capital','Lat','Lng','Total Persons'], axis=1)
District_Totals_plot  = District_Totals.plot(kind="bar",facecolor="green")
plt.title("Persons Per District")
#plt.ylim(0,30)
plt.xlabel("District")
plt.ylabel("Number of Persons (Thousands)")
plt.xticks(rotation=90)
#plt.tight_layout()
plt.legend(loc="best")
District_Totals_plot;